In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os

CLIENT_ID = input(" Enter your Spotify Client ID: ").strip()
CLIENT_SECRET = input(" Enter your Spotify Client Secret: ").strip()
REDIRECT_URI = "http://127.0.0.1:8889/callback/"  
PLAYLIST_NAME = input(" Enter your playlist name: ").strip()

sp_oauth = SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope="playlist-modify-private playlist-modify-public",
    open_browser=False
)

auth_url = sp_oauth.get_authorize_url()
print(f"\n Open this URL in browser and authorize:\n{auth_url}")

redirect_response = input("\n Paste the full redirect URL here:\n").strip()

code = sp_oauth.parse_response_code(redirect_response)
token_info = sp_oauth.get_access_token(code)
access_token = token_info["access_token"]

sp = spotipy.Spotify(auth=access_token)
user_id = sp.current_user()['id']
print(f"\n Logged in as: {sp.current_user()['display_name']}")

if not os.path.exists("playlist.txt"):
    print(" Error: playlist.txt not found.")
    raise SystemExit

with open("playlist.txt", "r", encoding="utf-8") as f:
    titles = [line.strip() for line in f if line.strip()]

if not titles:
    print(" Error: playlist.txt is empty.")
    raise SystemExit
playlist = sp.user_playlist_create(
    user=user_id,
    name=PLAYLIST_NAME,
    public=False,
    description="🪄 Synced from YouTube"
)
playlist_id = playlist["id"]
print(f"\n Playlist created: {PLAYLIST_NAME}")

track_uris = []
not_found = []

for title in titles:
    result = sp.search(q=title, type="track", limit=1)
    tracks = result.get("tracks", {}).get("items", [])
    if tracks:
        uri = tracks[0]["uri"]
        track_uris.append(uri)
        print(f" Found: {title}")
    else:
        not_found.append(title)
        print(f" Not found: {title}")

for i in range(0, len(track_uris), 100):
    sp.playlist_add_items(playlist_id, track_uris[i:i+100])

print(f"\n Added {len(track_uris)} track(s) to playlist '{PLAYLIST_NAME}'.")

if not_found:
    with open("not_found.txt", "w", encoding="utf-8") as nf:
        nf.write("\n".join(not_found))
    print(f" {len(not_found)} song(s) not found. Saved to 'not_found.txt'")
else:
    print(" All songs were found and added successfully!")